# Adding features to the dataset
After testing performances with 132 features, 15 new features are added, selecting: 
- min
- max
- mean
- median
- standard deviation

From the new features: 
- zero crossing rate
- rolloff
- onset strenght

I use the same class, where the get_feature methods has been changed.
Scaling is also applied, as it yielded better results in previous experiments. 

The extraction time went from about 26s per fold to 52s per fold.
The approach, and the code, is the same used in the dataset notebook.

## PCA

I also tried with PCA to check if this leads to better scores, 
I selected the number of features that exploit 0.99 variance.

In [ ]:
import sys
sys.path.append("..")
from src.data import Features
import pandas as pd
import numpy as np

In [ ]:
f = Features(save_path="../data/processed/extended",
             save_name="train_scaled_extended",
             folds=[1, 2, 3, 4, 6])

training_dataframe = f.get_dataframe()

In [ ]:
training_dataframe

The following code applies scaling and pca to the dataset. 
The two trained models are saved to reapply them to the test sets.

In [ ]:
scaled_df = f.scale_dataframe(training_dataframe,
                              save_path="../models/scalers/scaler_training_extended.pkl", 
                              save_scaler=True)

f.save_dataframe(scaled_df, save_name="train_extended")

reduced_df = f.select_features(scaled_df, 
                               n=0.99, 
                               save_pca=True)
f.save_dataframe(reduced_df, save_name="train_pca")

reduced_df

Scaling and pca is applied to the test sets and saved.

In [ ]:
for fold in [5, 7, 8, 9, 10]:
    print(f"Processing fold {fold}")
    
    f = Features(save_path="../data/processed/extended",
                 save_name=f"test_{fold}_extended",
                 folds=[fold])
    
    df = f.get_dataframe()
    
    scaled = f.apply_scaling(df, "../models/scalers/scaler_training_extended.pkl")
    f.save_dataframe(scaled, save_name=f"test_{fold}_extended")
    
    reduced = f.apply_pca(scaled, "../models/pca/pca_training.pkl")
    f.save_dataframe(reduced, save_name=f"test_{fold}_pca")

# Extracting image training set

In [ ]:
f = Features(save_path="../data/processed/cnn", 
             save_name="stft",
             folds=[1, 2, 3, 4, 6])

features, labels = f.get_save_image_training_set()

for fold in [5, 7, 8, 9, 10]:
    print(f"Processing fold {fold}")
    
    f = Features(save_path="../data/processed/cnn",
                 save_name=f"stft_test_{fold}",
                 folds=[fold])
    f.get_save_image_training_set()